# Code to Load Model, Preprocess Data, and Score New Leads (for Demo)

In [2]:
# ==============================================================================
# This cell demonstrates how to use the saved model and preprocessing tools
# in a NEW environment (like a different Colab notebook or a deployment script)
# to score new, unseen lead data.
# ==============================================================================

# STEP 1: Import Libraries
import pandas as pd
import joblib
import numpy as np
import re # For text cleaning


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

print("✅ Libraries imported.")

# STEP 2: Load the Trained Model and Preprocessing Tools

model_filename = 'final_knn_optimized_model.pkl'
scaler_filename = 'scaler.pkl'
label_encoders_filename = 'label_encoders.pkl'

loaded_model = None
loaded_scalers = None
loaded_label_encoders = None

try:
    loaded_model = joblib.load(model_filename)
    print(f"✅ Model loaded successfully from '{model_filename}'")
except FileNotFoundError:
    print(f"❌ Error: Model file '{model_filename}' not found. Please upload it.")
except Exception as e:
    print(f"❌ Error loading model: {e}")

try:
    loaded_scalers = joblib.load(scaler_filename)
    print(f"✅ Scalers loaded successfully from '{scaler_filename}'")
    loaded_work_exp_scaler = loaded_scalers['scaler']
    loaded_industry_size_scaler = loaded_scalers['scaler_industry_size']

except FileNotFoundError:
    print(f"❌ Error: Scaler file '{scaler_filename}' not found. Please upload it.")
except Exception as e:
    print(f"❌ Error loading scalers: {e}")


try:
    loaded_label_encoders = joblib.load(label_encoders_filename)
    print(f"✅ Label encoders loaded successfully from '{label_encoders_filename}'")


except FileNotFoundError:
    print(f"❌ Error: Label encoder file '{label_encoders_filename}' not found. Please upload it.")
except Exception as e:
    print(f"❌ Error loading label encoders: {e}")


# Proceed only if all artifacts are loaded
if loaded_model and loaded_scalers and loaded_label_encoders:
    print("\n✅ All necessary model and preprocessing artifacts loaded.")

    # STEP 3: Load New Data

    new_data_filename = 'not_converted_L_V1.xlsx' # Example filename

    try:
        new_leads_df = pd.read_excel(new_data_filename)
        print(f"\n✅ New data loaded successfully from '{new_data_filename}'")
        print("Shape of new data:", new_leads_df.shape)

        # STEP 4: Preprocess New Data using the Loaded Tools

        def convert_work_experience_years(exp):
            if pd.isna(exp) or exp == 'NA':
                return np.nan
            exp = str(exp).upper()
            if 'BELOW 4' in exp:
                return 2
            elif '4 TO 8' in exp:
                return 6
            elif '8 TO 12' in exp:
                return 10
            elif '12 TO 16' in exp:
                return 14
            elif '16 YEARS' in exp:
                return 18
            match_years = re.search(r'(\d+)\s*YEARS?', exp)
            if match_years:
                try:
                    return float(match_years.group(1))
                except ValueError:
                    return np.nan
            elif 'FRESHER' in exp:
                return 0
            return np.nan

        # Function to clean industry size (copy from training notebook)
        def clean_industry_size(val):
            if pd.isna(val):
                return np.nan
            val = str(val).replace(',', '').upper().strip()
            try:
                if 'BILLION' in val:
                    return float(val.split()[0])
                elif 'B' in val:
                    return float(val.replace('B', ''))
                elif 'MILLION' in val:
                    return float(val.split()[0]) / 1000
                elif 'M' in val:
                    return float(val.replace('M', '')) / 1000
                elif 'CR' in val:
                    return float(val.replace('₹', '').replace('CR', '')) * 0.12
                elif 'LAKH' in val:
                    return float(val.replace('₹', '').replace('LAKH', '')) * 0.00012
                else:
                    return float(val)
            except:
                return np.nan

        # Function to clean text (copy from training notebook)
        def clean_text(val):
            if pd.isna(val):
                return 'unknown'
            val = str(val).lower().strip()
            val = re.sub(r'[^\w\s]', '', val)
            return val

        # Apply cleaning and conversion
        new_leads_df['Work_Experience_Years'] = new_leads_df['Work Experience'].apply(convert_work_experience_years)
        new_leads_df['Industry_Size_Clean'] = new_leads_df['Industry Size (Billion)'].apply(clean_industry_size)
        new_leads_df['Job_Title_Clean'] = new_leads_df['Job Title'].apply(clean_text)
        new_leads_df['Industry_Clean'] = new_leads_df['Industry'].apply(clean_text)
        new_leads_df['Organization_Clean'] = new_leads_df['Organization'].apply(clean_text)
        new_leads_df['Publisher_Clean'] = new_leads_df['Publisher Name'].apply(clean_text)
        new_leads_df['Type_Clean'] = new_leads_df['Type'].apply(clean_text)

        print("✅ Cleaning and initial conversions applied to new data.")

        # Apply Label Encoding using LOADED encoders
        categorical_cols_clean = ['Job_Title_Clean', 'Industry_Clean', 'Organization_Clean', 'Publisher_Clean', 'Type_Clean']
        for col in categorical_cols_clean:
            if col in loaded_label_encoders:
                le = loaded_label_encoders[col]
                unseen_labels = set(new_leads_df[col].dropna()) - set(le.classes_) # Check non-null values
                if unseen_labels:
                    print(f"Warning: Unseen labels in column '{col}': {unseen_labels}. Attempting to map to 'unknown'.")
                    if 'unknown' in le.classes_:
                         # Replace unseen labels with 'unknown' before transforming
                         new_leads_df[col] = new_leads_df[col].apply(lambda x: 'unknown' if x in unseen_labels else x)
                    else:
                         pass # Already replaced with 'unknown' in clean_text if NaN

                if new_leads_df[col].isnull().any():
                    print(f"Warning: NaNs found in cleaned column '{col}' before encoding. Imputing with 'unknown'.")
                    new_leads_df[col].fillna('unknown', inplace=True) # Impute with 'unknown'


                new_leads_df[col + '_Encoded'] = le.transform(new_leads_df[col])
            else:
                print(f"❌ Error: Label encoder for '{col}' not found in loaded encoders.")
                new_leads_df[col + '_Encoded'] = np.nan # Cannot encode if encoder is missing


        print("✅ Label Encoding applied using loaded encoders.")


        loaded_work_exp_scaler = loaded_scalers['scaler']
        loaded_industry_size_scaler = loaded_scalers['scaler_industry_size']



        imputation_value = -999 # Using a distinct value to represent missingness

        # Scale Work Experience after imputation
        new_leads_df['Work_Exp_Scaled'] = loaded_work_exp_scaler.transform(new_leads_df[['Work_Experience_Years']].fillna(imputation_value))

        # Scale Industry Size after imputation
        new_leads_df['Industry_Size_Scaled'] = loaded_industry_size_scaler.transform(new_leads_df[['Industry_Size_Clean']].fillna(imputation_value))


        print("✅ Scaling applied using loaded scalers with imputation.")

        # Select the final features for prediction, ensuring names match the model's training features
        # The model was trained on the columns named 'Work_Exp_Scaled', 'Industry_Size_Scaled', etc.
        # We must ensure the DataFrame passed for prediction has these exact names.

        model_feature_cols_final = [
            'Work_Exp_Scaled', 'Industry_Size_Scaled', 'Job_Title_Clean_Encoded',
            'Industry_Clean_Encoded', 'Organization_Clean_Encoded', 'Publisher_Clean_Encoded',
            'Type_Clean_Encoded'
        ]

        # Select these specific columns for the prediction DataFrame
        # This ensures the column names are correct for the model
        X_new_predict = new_leads_df[model_feature_cols_final].copy()

        if X_new_predict.isnull().sum().sum() > 0:
             print("Warning: NaNs still present in prediction features. Imputing with placeholder.")
             # As a final safeguard, impute any remaining NaNs before prediction.
             # This shouldn't be necessary if the imputation logic is correct.
             X_new_predict.fillna(imputation_value, inplace=True)


        print("\n✅ New data preprocessed and ready for scoring.")
        print("Features for prediction (first 5 rows):")
        display(X_new_predict.head())
        print("Info on prediction features:")
        X_new_predict.info()


        # STEP 5: Generate Lookalike Scores and Probability Bands
        if X_new_predict.shape[0] > 0 and loaded_model:

            if hasattr(loaded_model, 'feature_names_in_'):
                 expected_features = loaded_model.feature_names_in_
                 X_new_predict = X_new_predict.reindex(columns=expected_features, fill_value=imputation_value)
                 print("✅ Ensured prediction features match model's expected order.")
            elif sorted(X_new_predict.columns) != sorted(model_feature_cols_final):
                 print("Warning: Prediction feature columns do not match expected columns or order. This could cause issues.")
                 # If feature_names_in_ is not available and columns don't match, prediction might fail or be incorrect.


            lookalike_scores = loaded_model.predict_proba(X_new_predict)[:, 1]

            # Attach scores back to the original new_leads_df using the index
            # Create a temporary DataFrame for scores with the same index as X_new_predict
            scores_df = pd.DataFrame({'Lookalike_Score': lookalike_scores}, index=X_new_predict.index)

            new_leads_scored_df = new_leads_df.merge(scores_df, left_index=True, right_index=True, how='left')


            # Define the probability band function
            def assign_band(score):
                if pd.isna(score) or score == imputation_value: # Handle cases where scoring might have failed or imputed
                    return 'Not Scored'
                if score >= 0.80:
                    return "Very High"
                elif score >= 0.60:
                    return "High"
                elif score >= 0.40:
                    return "Medium"
                else:
                    return "Low"

            # Apply the function to create the 'Probability_Band' column
            new_leads_scored_df['Probability_Band'] = new_leads_scored_df['Lookalike_Score'].apply(assign_band)

            print("\n✅ Lookalike scores and probability bands generated.")

            # Step: Sort and get top 10 lookalikes for display from the new data
            # Filter out leads that couldn't be scored if imputation failed or data was missing
            df_top_lookalikes_sorted_new = new_leads_scored_df[new_leads_scored_df['Probability_Band'] != 'Not Scored'].sort_values(by='Lookalike_Score', ascending=False).head(10)

            print("\n🎯 Top 10 Lookalike Leads (Scored from New Data):")
            # Display key columns for top 10 display
            display(df_top_lookalikes_sorted_new[[
                'Registered Name', 'Registered Mobile', 'Lookalike_Score', 'Probability_Band',
                # Include original columns used in preprocessing for verification
                'Work Experience', 'Industry Size (Billion)', 'Job Title', 'Industry', 'Organization', 'Publisher Name', 'Type'
            ]])

            # Step: Count the number of leads in each probability band for the new data
            band_counts_new_data = new_leads_scored_df['Probability_Band'].value_counts().reindex(['Very High', 'High', 'Medium', 'Low', 'Not Scored'])

            print("\n✅ Distribution of Scored New Leads by Probability Band:")
            print(band_counts_new_data)

            print(f"\nTotal processed leads: {new_leads_scored_df.shape[0]}")
            print(f"Total scored leads: {band_counts_new_data.sum()}")


            # STEP 6: Export the Scored New Dataset to Excel
            all_scored_new_leads_filename = "All_New_Lookalike_Leads_Scored.xlsx"
            new_leads_scored_df.to_excel(all_scored_new_leads_filename, index=False)
            print(f"\n✅ Complete scored new leads exported to: {all_scored_new_leads_filename}")

            # Optional: Export the priority leads as well (Very High and High)
            priority_leads_filename = "Priority_New_Leads.xlsx"
            df_priority_new = new_leads_scored_df[new_leads_scored_df['Probability_Band'].isin(['Very High', 'High'])]
            df_priority_new.to_excel(priority_leads_filename, index=False)
            print(f"✅ Priority (Very High & High) new leads exported to: {priority_leads_filename}")

            # Step 7: Download the exported files (as requested)
            from google.colab import files
            import os

            print("\nAttempting to download exported files...")

            if os.path.exists(all_scored_new_leads_filename):
                try:
                    files.download(all_scored_new_leads_filename)
                    print(f"'{all_scored_new_leads_filename}' download initiated.")
                except Exception as e:
                    print(f"Error downloading '{all_scored_new_leads_filename}': {e}")
            else:
                print(f"'{all_scored_new_leads_filename}' not found for download.")

            if os.path.exists(priority_leads_filename):
                 try:
                     files.download(priority_leads_filename)
                     print(f"'{priority_leads_filename}' download initiated.")
                 except Exception as e:
                     print(f"Error downloading '{priority_leads_filename}': {e}")
            else:
                 print(f"'{priority_leads_filename}' not found for download.")


        else:
            print("\n❌ No valid data available for scoring after preprocessing or model not loaded.")


    except FileNotFoundError:
        print(f"❌ Error: New data file '{new_data_filename}' not found. Please upload it.")
    except Exception as e:
        print(f"❌ Error processing new data: {e}")

else:
    print("\n❌ Model or preprocessing artifacts not loaded. Cannot proceed with scoring.")

✅ Libraries imported.
✅ Model loaded successfully from 'final_knn_optimized_model.pkl'
✅ Scalers loaded successfully from 'scaler.pkl'
✅ Label encoders loaded successfully from 'label_encoders.pkl'

✅ All necessary model and preprocessing artifacts loaded.

✅ New data loaded successfully from 'not_converted_L_V1.xlsx'
Shape of new data: (28320, 33)
✅ Cleaning and initial conversions applied to new data.
✅ Label Encoding applied using loaded encoders.
✅ Scaling applied using loaded scalers with imputation.

✅ New data preprocessed and ready for scoring.
Features for prediction (first 5 rows):


,Work_Exp_Scaled,Industry_Size_Scaled,Job_Title_Clean_Encoded,Industry_Clean_Encoded,Organization_Clean_Encoded,Publisher_Clean_Encoded,Type_Clean_Encoded
0,-0.893869,-10.260959,7600,89,9926,49,2
1,-193.934609,-0.506690,584,20,2116,10,1
2,-193.934609,-10.260959,7600,89,9926,7,2
3,-193.934609,-10.260959,5188,89,5193,10,2
4,-193.934609,-10.260959,1624,89,10643,10,2


Info on prediction features:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28320 entries, 0 to 28319
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Work_Exp_Scaled             28320 non-null  float64
 1   Industry_Size_Scaled        28320 non-null  float64
 2   Job_Title_Clean_Encoded     28320 non-null  int64  
 3   Industry_Clean_Encoded      28320 non-null  int64  
 4   Organization_Clean_Encoded  28320 non-null  int64  
 5   Publisher_Clean_Encoded     28320 non-null  int64  
 6   Type_Clean_Encoded          28320 non-null  int64  
dtypes: float64(2), int64(5)
memory usage: 1.5 MB
✅ Ensured prediction features match model's expected order.

✅ Lookalike scores and probability bands generated.

🎯 Top 10 Lookalike Leads (Scored from New Data):


,Registered Name,Registered Mobile,Lookalike_Score,Probability_Band,Work Experience,Industry Size (Billion),Job Title,Industry,Organization,Publisher Name,Type
18065,KIRAN KUMAR,`+91-9945644664,1.0,Very High,8 TO 12 YEARS,NaN,TEAM LEAD,NaN,MEERITT,TIMES OF INDIA,NaN
13092,SANDEEP PATIL,`+91-9545251155,1.0,Very High,BELOW 4 YEARS,NaN,APPLICATION ENGINEER,NaN,GALAXY MACHINERY,GOOGLE ADS,NaN
24908,ASHOK RAJ,`+91-9901977227,1.0,Very High,12 TO 16 YEARS,NaN,TECH ARCHITECT,NaN,LEGATO,EVENT,NaN
18067,MALIKARJUNA B S,`+91-9880597818,1.0,Very High,16 YEARS AND ABOVE,NaN,ENGINEERING MANAGER,NaN,NXP SEMICONDUCTORS,TELEPHONY,NaN
18074,BATTULAKAMALAKAR,`+91-9014221961,1.0,Very High,BELOW 4 YEARS,NaN,IMPULSE,NaN,MARS,REFERRAL,NaN
4454,SAMPATH KUMAR,`+91-9886008554,1.0,Very High,NaN,NaN,PROJECT LEAD,NaN,INTEVA AUTOMOTIVE PRIVATE LIMITED,FACEBOOK,NaN
18078,VENKATESH SOMA,`+91-7835958289,1.0,Very High,NaN,NaN,TEAM LEAD MEDICAL DEVICE REGULATORY AFFAIRS,NaN,CLINCHOICE,LINKEDIN,NaN
4460,PINAKI GHOSH,`+91-9513157866,1.0,Very High,NaN,NaN,TEACHER,NaN,CANADIAN INTERNATIONAL SCHOOL,FACEBOOK,NaN
24891,RAVIDAS BHAT,`+91-9443374201,1.0,Very High,16 YEARS AND ABOVE,NaN,SR MANAGER CENTRE OF EXCELLANCE,NaN,PRECISION ENGINEERING DIVISION-TITAN COMPANY L...,FMA DIGITAL,NaN
4468,KUSUMA G S,`+91-8792019731,1.0,Very High,BELOW 4 YEARS,NaN,FRESHER,NaN,NOT YET WORKING,FACEBOOK,NaN



✅ Distribution of Scored New Leads by Probability Band:
Probability_Band
Very High      2106.0
High           2182.0
Medium            NaN
Low           24032.0
Not Scored        NaN
Name: count, dtype: float64

Total processed leads: 28320
Total scored leads: 28320.0

✅ Complete scored new leads exported to: All_New_Lookalike_Leads_Scored.xlsx
✅ Priority (Very High & High) new leads exported to: Priority_New_Leads.xlsx

Attempting to download exported files...


...

<IPython.core.display.Javascript object>

'All_New_Lookalike_Leads_Scored.xlsx' download initiated.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'Priority_New_Leads.xlsx' download initiated.
